Goal of this notebook: given a table-like grid of cells, write an equivalent Python program that can be run on the input data (cells without any dependencies) to generate the output (cells in the last column or cells without any dependencies).

First example: one single row.

In [5]:
def translate(table):
    ...
    
table = [['3', '4', '=A1+A2']]

# Equivalent program:
def f_2(x, y):
    return x + y  # Translation of 'A1+A2'

def row_function(table):
    ...

def input_columns(table):
    ...

def output_columns(table):
    ...
    
row_function(table) == f_2  # More like AST of f_2 but w/e
input_columns(table) == [0, 1]
output_columns(table) == [2]

# To evaluate:
# for row in table:
#    table[output_columns] = row_function(table)(*[row[col] for col in input_columns(table)]))r
None